# PAGE hm3 analysis

### Step 2: Calculate GRM
Calculate GRM matrices
```

### Step 3: estimate parameters
```bash
for n_indiv in 5000 10000 20000; do
    for anc_prop in 0.2 0.5; do
        dset_prefix=n_indiv_${n_indiv}_anc_prop_${anc_prop}
        for cor in 0.5 0.8 1.0; do
            qsub estimate_HE.sh ${dset_prefix} ${cor}
        done
    done
done
```

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import xarray as xr
import numpy as np
import admix
import matplotlib.pyplot as plt
from utils import *
import dask
import dask.array as da
from tqdm import tqdm

def quantile_normalize(val):
    from scipy.stats import rankdata, norm
    return norm.ppf((rankdata(val) - 0.5) / len(val)) 

In [9]:
dset = load_hm3()

100%|██████████| 22/22 [00:00<00:00, 57.94it/s]


In [10]:
K1 = np.load("out/admix_grm/K1.all.npy")
K2 = np.load("out/admix_grm/K2.all.npy")
K12 = np.load("out/admix_grm/K12.all.npy")

In [11]:
dset["A1"] = (("indiv", "indiv"), K1 + K2)
dset["A2"] = (("indiv", "indiv"), K12 + K12.T)

In [1]:
trait_list = [
    # Inflammtory traits
    "crp",
    "total_wbc_cnt",
    "mean_corp_hgb_conc",
    "platelet_cnt",
    # lipid traits
    "hdl",
    "ldl",
    "triglycerides",
    "total_cholesterol",
    # lifestyle traits
    "cigs_per_day_excl_nonsmk_updated",
    "coffee_cup_day",
    # glycemic traits
    "a1c",
    "insulin",
    "glucose",
    "t2d_status",
    # electrocardiogram traits
    "qt_interval",
    "qrs_interval",
    "pr_interval",
    # blood pressure traits
    "systolic_bp",
    "diastolic_bp",
    "hypertension",
    # anthropometric traits
    "waist_hip_ratio",
    "height",
    "bmi",
    # kidney traits
    "egfrckdepi",
]


In [9]:
with open("trait_list.txt", 'w') as f:
    f.writelines('\n'.join(trait_list))

In [137]:
dict_result = {"trait": [], 
             "n_indiv": [], 
             "est": [],
             "est_var": []}

for trait in tqdm(trait_list):
    dset_cor = dset.isel(indiv=~np.isnan(dset[f"{trait}@indiv"].values))
    study_dummies = pd.get_dummies(dset_cor["study@indiv"], drop_first=True)
    for c in study_dummies:
        dset_cor[f"study_dummy_{c}@indiv"] = ("indiv", study_dummies[c])
    study_dummy_cols = [f"study_dummy_{c}" for c in study_dummies]
    
    pheno = dset_cor[f"{trait}@indiv"].values
#     pheno = (pheno - pheno.mean()) / pheno.std()
    pheno = quantile_normalize(pheno)
    est, est_var = admix.estimate.admix_gen_cor(dset=dset_cor, 
                                                pheno=pheno, 
                                                cov_cols=["age", "sex"] + study_dummy_cols + [f"geno_EV{i}" for i in range(1, 11)])[0]
    print('--------------')
    print(f"{trait} (N={dset_cor.dims['indiv']})")
    print(est)
    print(est_var)
    
    dict_result["trait"].append(trait)
    dict_result["n_indiv"].append(dset_cor.dims['indiv'])
    dict_result["est"].append(est)
    dict_result["est_var"].append(est_var)

  0%|          | 0/24 [00:00<?, ?it/s]/u/project/pasaniuc/kangchen/software/miniconda3/lib/python3.8/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
  4%|▍         | 1/24 [00:20<07:58, 20.82s/it]

--------------
crp (N=8521)
[0.42140723 0.39212931 0.82550683]
[[ 0.00601432  0.00846136 -0.00176934]
 [ 0.00846136  0.01635791 -0.00238153]
 [-0.00176934 -0.00238153  0.00074112]]


  8%|▊         | 2/24 [00:43<08:03, 21.99s/it]

--------------
total_wbc_cnt (N=8889)
[ 0.64724382 -1.05345162  0.65345385]
[[ 0.03220871  0.03939174 -0.00999912]
 [ 0.03939174  0.15508984 -0.01077568]
 [-0.00999912 -0.01077568  0.00334822]]


 12%|█▎        | 3/24 [00:47<04:51, 13.87s/it]

...

--------------
height (N=17286)
[0.39970117 0.38581886 0.47209472]
[[ 0.00104545  0.00128381 -0.00029493]
 [ 0.00128381  0.0032907  -0.00032907]
 [-0.00029493 -0.00032907  0.00012583]]


 96%|█████████▌| 23/24 [14:07<01:17, 77.82s/it]

--------------
bmi (N=17264)
[0.37357099 0.41306849 0.8248541 ]
[[ 0.00157557  0.00213294 -0.00044475]
 [ 0.00213294  0.00424611 -0.0005712 ]
 [-0.00044475 -0.0005712   0.00022924]]


100%|██████████| 24/24 [14:26<00:00, 36.10s/it]

--------------
egfrckdepi (N=8261)
[0.14721418 0.12131103 0.68707128]
[[ 0.00322823  0.00472596 -0.00097069]
 [ 0.00472596  0.00834396 -0.00138296]
 [-0.00097069 -0.00138296  0.00042424]]


In [138]:
df_rls = dict()
for name in ["trait", "n_indiv"]:
    df_rls[name] = dict_result[name]

for i, name in enumerate(["sigma_g", "gamma", "sigma_e"]):
    df_rls[name] = [f"{e[i]:.2f} ({np.sqrt(v[i, i]):.2f})" for e, v in zip(dict_result["est"], dict_result["est_var"])]
df_rls = pd.DataFrame(df_rls)

In [ ]:
df_rls

In [142]:
df_rls.to_csv("result/page.csv", index=False)